In [ ]:
!pip install auditok
!pip install noisereduce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import glob
import numpy as np
from scipy.io import wavfile
from scipy.io.wavfile import write
import noisereduce as nr
import auditok
import pandas as pd
import datetime
import os
import statistics

In [ ]:
os.getcwd()

'/content'

Once the drive is mounted, make folder named 'data' in the content folder and store the files you want to run the pipeline on to the 'data' folder.

In [ ]:
#extracts path for .wav files
file_list = glob.glob('/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/*.wav')
file_list

['/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_103949.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_113907.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_123902.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_133902.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_143902.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_153902.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_163902.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_173902.wav',
 '/content/drive/MyDrive/Cat Calls Triangulation/0131 Outside Leopard 3:20-3:27/SMM10131_20240320_183902.wav',
 

In [ ]:
for file in file_list:
  filename = file[-28:-4]
  print(filename)

31 Outside Leopard 3:20-


In [ ]:
# iterates over .wav files and eliminates background noise
# saves modified audio in the following format --> filename_reduced.wav
for file in file_list:
    rate, data = wavfile.read(file)
    filename = file[-28:-4]
    directory = "data/"
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    new_name = filename + "_reduced.wav"
    write(directory+new_name, rate, reduced_noise)


<ipython-input-7-9b3aee546046>:4: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, data = wavfile.read(file)


In [ ]:
#extracts path for .wav files that have no background noise
audio_list = glob.glob('data/*_reduced.wav')
audio_list

['data/SMM10131_20240320_103949_reduced.wav']

In [ ]:
# for each .wav file, extracts regions of interest and saves them as clips
def audio_pipeline(file, dataset):
    filename = file[5:-4]
    #os.mkdir("data//"+filename)
    os.makedirs("data//"+filename,exist_ok=True)
    audio_regions = auditok.split(file,
                                  min_dur=0.2,        # minimum duration of a valid audio event in seconds
                                  max_dur=0.5,        # maximum duration of an event
                                  max_silence=0.3,    # maximum duration of tolerated continuous silence within an event
                                  energy_threshold=60 # threshold of detection
                                 )

    average_n_calls = 0
    curr = 5
    curr_count = 0
    count_list = []

    for i, r in enumerate(audio_regions):
        #print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))
        region = i
        start = str(datetime.timedelta(seconds=r.meta.start))
        end = str(datetime.timedelta(seconds=r.meta.end))

        #

        dur = r.meta.end - r.meta.start
        if dur < 0.04:
          print(f"Skipped, duration = {dur}")
          continue

        #

        if r.meta.start < curr:
          curr_count += 1
        else:
          curr += 5
          count_list.append(curr_count)
          curr_count = 1

        #

        dataset.append([filename[:-8], i, start[:-4], end[:-4]])
        r.save("data//" + filename + "//region_{meta.start:.3f}-{meta.end:.3f}.wav")

    print(f"{filename[:-8]}")
    print(f"Avg/5sec : {statistics.fmean(count_list)}")
    print(f"Total no. of Detected Saw Calls: {sum(count_list)}")
    print(" ")

        #347

In [ ]:
dataset = []
# iterating over all noise reduced files and applying the above function on each
for file in audio_list:
    audio_pipeline(file, dataset)

SMM10131_20240320_103949
Avg/5sec : 1.198501872659176
Total no. of Detected Saw Calls: 320
 


In [ ]:
# saves results into an excel file
df = pd.DataFrame(dataset, columns=['File', 'Region', 'Start Time', 'End Time'])
df.to_excel("data//output.xlsx")